In [3]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version:{}".format(sys.version))
print("Tensorflow version:{}".format(tf.__version__))

System version:3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version:2.6.1


In [13]:
import numpy as np

In [7]:
!pip install keras==2.6

  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)


In [10]:
!pip install protobuf==3.20.* --user

In [3]:
!pip install scrapbook

  Using cached scrapbook-0.5.0-py3-none-any.whl (34 kB)
  Using cached papermill-2.4.0-py3-none-any.whl (38 kB)
  Using cached ansiwrap-0.8.4-py2.py3-none-any.whl (8.5 kB)
  Using cached textwrap3-0.9.2-py2.py3-none-any.whl (12 kB)


In [9]:
!pip install tensorflow==2.6.1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.6.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.14.0rc0, 2.14.0rc1)
ERROR: No matching distribution found for tensorflow==2.6.1


In [ ]:
!pip install tensorflow

In [8]:
!pip install numpy==1.20.3 --user

  Using cached numpy-1.20.3-cp38-cp38-win_amd64.whl (13.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.


In [1]:
conda uninstall numpy

Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://rep

In [1]:
!pip show numpy

Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\super\appdata\roaming\python\python38\site-packages
Requires: 
Required-by: Bottleneck, category-encoders, contourpy, cornac, h5py, Keras-Preprocessing, lightfm, lightgbm, matplotlib, numba, opt-einsum, pandas, pandera, patsy, powerlaw, pyarrow, recommenders, scikit-learn, scikit-surprise, scipy, seaborn, statsmodels, tensorboard, tensorflow, transformers


In [4]:
epochs=1
seed=40
batch_size=64

MIND_type='demo'

In [5]:

data_path = r"C:\Users\super\OneDrive\Documents\SSIS\Swoopo"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

In [49]:
tmpdir = TemporaryDirectory()
data_path = r"C:\Users\super\OneDrive\Documents\SSIS\Swoopo"

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:01<00:00, 8.76kKB/s]
100%|██████████| 9.84k/9.84k [00:02<00:00, 4.46kKB/s]
100%|██████████| 95.0k/95.0k [00:20<00:00, 4.74kKB/s]


In [6]:
print(yaml_file)

C:\Users\super\OneDrive\Documents\SSIS\Swoopo\utils\lstur.yaml


In [7]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 1, 'batch_size': 64, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\word_dict.pkl', 'userDict_file': 'C:\\Users\\super\\OneDrive\\Documents\\SSIS\\Swoopo\\utils\\uid2index.pkl'}


In [8]:
iterator = MINDIterator

In [9]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(None, 400), dtype=float32)


C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [11]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

37it [00:01, 25.28it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

{'group_auc': nan, 'mean_mrr': nan, 'ndcg@5': nan, 'ndcg@10': nan}



C:\Users\super\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\super\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

1it [00:14, 14.14s/it]
37it [00:02, 17.94it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

at epoch 1
train info: logloss loss:1.6309136152267456
eval info: group_auc:nan, mean_mrr:nan, ndcg@10:nan, ndcg@5:nan
at epoch 1 , train time: 14.1 eval time: 2.1
CPU times: total: 22 s
Wall time: 16.2 s


In [14]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

37it [00:02, 15.34it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

{'group_auc': nan, 'mean_mrr': nan, 'ndcg@5': nan, 'ndcg@10': nan}
CPU times: total: 3.83 s
Wall time: 2.43 s


In [15]:
sb.glue("res_syn", res_syn)

C:\Users\super\anaconda3\envs\python38\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [16]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "lstur_ckpt"))

In [17]:
model.model.save_weights("lstur_ckpt.h5")

In [18]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

37it [00:04,  7.70it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [19]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

0it [00:00, ?it/s]


In [48]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

In [52]:
import pickle

In [53]:
with open(userDict_file, 'rb') as f:
    s = pickle.load(f)

In [54]:
s

{'U8210': 1,
 'U89928': 2,
 'U52040': 3,
 'U77254': 4,
 'U57621': 5,
 'U47706': 6,
 'U48649': 7,
 'U28154': 8,
 'U7012': 9,
 'U81910': 10,
 'U26507': 11,
 'U39957': 12,
 'U39072': 13,
 'U27564': 14,
 'U32371': 15,
 'U506': 16,
 'U44141': 17,
 'U135': 18,
 'U82397': 19,
 'U200': 20,
 'U50599': 21,
 'U37610': 22,
 'U10439': 23,
 'U4267': 24,
 'U80877': 25,
 'U85108': 26,
 'U52590': 27,
 'U41342': 28,
 'U52060': 29,
 'U31908': 30,
 'U32074': 31,
 'U40220': 32,
 'U33683': 33,
 'U42378': 34,
 'U18189': 35,
 'U63261': 36,
 'U88995': 37,
 'U71678': 38,
 'U68407': 39,
 'U55648': 40,
 'U72590': 41,
 'U92428': 42,
 'U67305': 43,
 'U33152': 44,
 'U61968': 45,
 'U22486': 46,
 'U2273': 47,
 'U51693': 48,
 'U50355': 49,
 'U32191': 50,
 'U91182': 51,
 'U2509': 52,
 'U34502': 53,
 'U16137': 54,
 'U50098': 55,
 'U72391': 56,
 'U58699': 57,
 'U51569': 58,
 'U67407': 59,
 'U65002': 60,
 'U10396': 61,
 'U65542': 62,
 'U12433': 63,
 'U654': 64,
 'U67486': 65,
 'U4480': 66,
 'U57613': 67,
 'U48187': 68,
 'U

In [55]:
with open(userDict_file+"2", 'wb') as f:
    pickle.dump(s, f)